In [0]:
import requests

# The 'pp-complete.csv' is the official 6GB file required for your assessment
full_data_url = "http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-complete.csv"
full_dest_path = "/Volumes/workspace/default/uk_land_registry/uk_property_full.csv"

try:
    print("Starting BIG DATA download (6GB). This may take 5-10 minutes...")
    with requests.get(full_data_url, stream=True) as r:
        r.raise_for_status()
        with open(full_dest_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024*1024):
                f.write(chunk)
    print(f"SUCCESS! 6GB Dataset is now ready: {full_dest_path}")
    
    # Verify the count—this should be over 20 million rows
    full_df = spark.read.csv(full_dest_path, schema=schema)
    print(f"Total Big Data Rows: {full_df.count()}")
    
except Exception as e:
    print(f"Full download failed: {e}")

Starting BIG DATA download (6GB). This may take 5-10 minutes...
SUCCESS! 6GB Dataset is now ready: /Volumes/workspace/default/uk_land_registry/uk_property_full.csv
Full download failed: name 'schema' is not defined


In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Re-declaring the path to your Volume
full_dest_path = "/Volumes/workspace/default/uk_land_registry/uk_property_full.csv"

# Re-defining the Schema for 30 million rows
schema = StructType([
    StructField("Transaction_ID", StringType(), True),
    StructField("Price", IntegerType(), True),
    StructField("Date", StringType(), True),
    StructField("Postcode", StringType(), True),
    StructField("Property_Type", StringType(), True),
    StructField("Old_New", StringType(), True),
    StructField("Duration", StringType(), True),
    StructField("PAON", StringType(), True),
    StructField("SAON", StringType(), True),
    StructField("Street", StringType(), True),
    StructField("Locality", StringType(), True),
    StructField("Town_City", StringType(), True),
    StructField("District", StringType(), True),
    StructField("County", StringType(), True),
    StructField("PPD_Category", StringType(), True),
    StructField("Record_Status", StringType(), True)
])

# Loading the data
full_path = "/Volumes/workspace/default/uk_land_registry/uk_property_full.csv"
property_df = spark.read.format("csv").schema(schema).load(full_path)

print(f"Connection Restored! Total Rows: {property_df.count()}")

Connection Restored! Total Rows: 30906560


In [0]:
from pyspark.sql.functions import col, to_timestamp

# 1. Clean the Data
silver_df = property_df.select(
    col("Price").cast("int"),
    to_timestamp(col("Date"), "yyyy-MM-dd HH:mm").alias("Sale_Date"),
    col("Postcode"),
    col("Property_Type"),
    col("Old_New"),
    col("Town_City"),
    col("District")
).dropna() # Remove rows with missing values to ensure ML accuracy

# 2. Show the clean version
print(f"Cleaned Data Count: {silver_df.count()}")
silver_df.show(5)

Cleaned Data Count: 30856185
+-----+-------------------+--------+-------------+-------+---------+--------------------+
|Price|          Sale_Date|Postcode|Property_Type|Old_New|Town_City|            District|
+-----+-------------------+--------+-------------+-------+---------+--------------------+
|36995|1995-03-24 00:00:00|SE19 3NF|            F|      N|   LONDON|             CROYDON|
|25000|1995-03-31 00:00:00| E16 1LG|            F|      N|   LONDON|              NEWHAM|
|25500|1995-05-17 00:00:00| EN3 6EA|            F|      N|  ENFIELD|             ENFIELD|
|42000|1995-04-21 00:00:00| N13 4RS|            T|      N|   LONDON|             ENFIELD|
|43000|1995-06-30 00:00:00|RM10 7NU|            T|      N| DAGENHAM|BARKING AND DAGENHAM|
+-----+-------------------+--------+-------------+-------+---------+--------------------+
only showing top 5 rows


In [0]:
from pyspark.ml.feature import StringIndexer, VectorAssembler

# 1. Convert 'Property_Type' (D, S, T, etc.) into numbers
indexer = StringIndexer(inputCol="Property_Type", outputCol="PropertyIndex")
indexed_df = indexer.fit(property_df).transform(property_df)

# 2. Vectorization (The Euclidean Prep)
# We combine the Price and the new PropertyIndex into a 'features' vector
assembler = VectorAssembler(inputCols=["Price", "PropertyIndex"], outputCol="features")
ml_data = assembler.transform(indexed_df).select("features", "PropertyIndex")

ml_data.show(5)

+-------------+-------------+
|     features|PropertyIndex|
+-------------+-------------+
|[36995.0,3.0]|          3.0|
|[25000.0,3.0]|          3.0|
|[25500.0,3.0]|          3.0|
|[42000.0,0.0]|          0.0|
|[43000.0,0.0]|          0.0|
+-------------+-------------+
only showing top 5 rows


In [0]:
from pyspark.ml.feature import StringIndexer, VectorAssembler

# 1. Indexing: Convert text categories to numeric indices
indexer = StringIndexer(inputCol="Property_Type", outputCol="label")
indexed_df = indexer.fit(property_df).transform(property_df)

# 2. Vectorization: Combine Price and other features into one 'features' column
# This is a key Technical Requirement for your project
assembler = VectorAssembler(inputCols=["Price"], outputCol="features")
final_data = assembler.transform(indexed_df).select("features", "label")

final_data.show(5)

+---------+-----+
| features|label|
+---------+-----+
|[36995.0]|  3.0|
|[25000.0]|  3.0|
|[25500.0]|  3.0|
|[42000.0]|  0.0|
|[43000.0]|  0.0|
+---------+-----+
only showing top 5 rows


In [0]:
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier, RandomForestClassifier
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, ClusteringEvaluator

# 1. Split Data: 80% for training, 20% for testing (Standard Big Data practice)
train_df, test_df = final_data.randomSplit([0.8, 0.2], seed=42)

# 2. Initialize the Algorithms
lr = LogisticRegression(labelCol="label", featuresCol="features")
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features")
rf = RandomForestClassifier(labelCol="label", featuresCol="features")
km = KMeans(k=5, seed=1) # Searching for 5 price-based property clusters

# 3. The "Training Factory"
# This is where your 30.9 million rows are processed in parallel
lr_model = lr.fit(train_df)
dt_model = dt.fit(train_df)
rf_model = rf.fit(train_df)
km_model = km.fit(final_data)

# 4. Evaluate Accuracy (Distinction Criteria)
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")

lr_acc = evaluator.evaluate(lr_model.transform(test_df))
dt_acc = evaluator.evaluate(dt_model.transform(test_df))
rf_acc = evaluator.evaluate(rf_model.transform(test_df))

print(f"Logistic Regression Accuracy: {lr_acc}")
print(f"Decision Tree Accuracy: {dt_acc}")
print(f"Random Forest Accuracy: {rf_acc}")

Logistic Regression Accuracy: 0.35135234359104905
Decision Tree Accuracy: 0.3715625352775089
Random Forest Accuracy: 0.3714354149476041


In [0]:
# Stability Test: Train on a different 80/20 split to see if accuracy stays the same
train_df_2, test_df_2 = final_data.randomSplit([0.8, 0.2], seed=123)
stability_model = dt.fit(train_df_2)
stability_acc = evaluator.evaluate(stability_model.transform(test_df_2))

print(f"Original Accuracy: {dt_acc}")
print(f"Stability Test Accuracy: {stability_acc}")
print(f"Variance: {abs(dt_acc - stability_acc)}")

Original Accuracy: 0.3715625352775089
Stability Test Accuracy: 0.37209474185670677
Variance: 0.0005322065791978914


In [0]:
# The 'Hard Reset' Load
full_path = "/Volumes/workspace/default/uk_land_registry/uk_property_full.csv"

try:
    # We use a direct spark read with no extra prefixes
    property_df = spark.read.format("csv").schema(schema).load(full_path)
    
    # We trigger an 'action' to force Spark to actually find the file
    print(f"Searching for file... Found! Total Rows: {property_df.count()}")
    
except Exception as e:
    print(f"Still failing. The system says: {e}")

Searching for file... Found! Total Rows: 30906560


In [0]:
from pyspark.sql import functions as F

# 1. Create the GOLD Layer
# We reduce 30M rows into a few thousand city-level summary points
gold_df = property_df.groupBy("Town_City", "Property_Type", "Old_New") \
    .agg(F.avg("Price").alias("Avg_Price"), 
         F.count("*").alias("Total_Sales"),
         F.max("Price").alias("Max_Price"))

# 2. Export to your Volume (The Coalesce Nuance)
# .coalesce(1) ensures it's one file for easy download to your HP Omen
gold_df.coalesce(1).write.mode("overwrite").option("header", "true").csv("/Volumes/workspace/default/uk_land_registry/gold_tableau_data")

# 3. Get the Final Statistics for your Report
avg_price = property_df.select(F.avg("Price")).collect()[0][0]

print("GOLD Layer Created successfully!")
print(f"Final National Average: £{avg_price:,.2f}")

GOLD Layer Created successfully!
Final National Average: £234,011.17


In [0]:
# Technical Requirement 1a: Creating a data sample for the repository
# We take only 100 rows so the file size is tiny (KBs)
sample_df = silver_df.limit(100)
sample_df.coalesce(1).write.mode("overwrite").option("header", "true").csv("/Volumes/workspace/default/uk_land_registry/data_sample")

print("Sample created. Download this CSV and put it in your 'data/samples/' folder.")

Sample created. Download this CSV and put it in your 'data/samples/' folder.
